In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

2023-05-18 14:22:30.788167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("training_set.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head()
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)

In [3]:
df['domain1_score']=temp['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,7,Patience is when your waiting .I was patience ...,5
1,2,7,"I am not a patience person, like I canât sit...",4
2,3,7,One day I was at basketball practice and I was...,5
3,4,7,I going to write about a time when I went to t...,6
4,5,7,It can be very hard for somebody to be patient...,4


In [4]:
df['essay'][0]

'Patience is when your waiting .I was patience when in line waiting for lunch .I didnâ\x80\x99t c ut any one to eat .I was standing and waiting for my turn .Patience ,some people donâ\x80\x99t have it .Lots of people just cut or yell at you because they donâ\x80\x99t have  any patience. Sometimes people will push you out of their way .They only do that because they donâ\x80\x99t have patience at all. Patience is what people need .People need patience because lots o f feelings get hurt .Everyone should have patience.'

In [5]:
temp.head()

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,7,Patience is when your waiting .I was patience ...,5,Patience waiting I patience line waiting lunch...,382,94,3,4.063830,0,24,26,2,5
1,2,7,"I am not a patience person, like I canât sit...",4,I patience person like I cant sit sit five mi...,348,94,3,3.702128,7,22,18,4,7
2,3,7,One day I was at basketball practice and I was...,5,One day I basketball practice I running team I...,577,152,1,3.796053,9,29,39,9,6
3,4,7,I going to write about a time when I went to t...,6,I going write time I went fair fun saw ride ...,840,222,13,3.783784,3,45,55,9,14
4,5,7,It can be very hard for somebody to be patient...,4,It hard somebody patient If patient understa...,629,154,12,4.084416,1,22,37,11,17


In [6]:
#Make Dataset
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
X_train.shape

(1098, 3)

**PREPROCESSING**

In [9]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [10]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)


In [11]:
len(train_sents)

12608

In [12]:
train_sents[0]

['Patience', 'way']

**Preparing WORD2VEC and LSTM Model**

In [13]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [14]:
#Training Word2Vec model
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 vector_size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

/var/folders/kn/gnm3d9hn21d8t3q08syhwsdw0000gp/T/ipykernel_7028/1462232201.py:15: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [15]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model.wv.get_vector(i))        
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)

In [16]:
training_vectors.shape

(1098, 300)

In [17]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

2023-05-18 14:24:58.331464: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 300)            721200    
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [18]:
training_vectors.shape

(1098, 1, 300)

**TRAINING AND PREDICTION**

In [19]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=150)

Epoch 1/150
18/18 [==============================] - 4s 11ms/step - loss: 16.0870 - mae: 3.4939
Epoch 2/150
18/18 [==============================] - 0s 11ms/step - loss: 3.0387 - mae: 1.4300
Epoch 3/150
18/18 [==============================] - 0s 11ms/step - loss: 2.7958 - mae: 1.3488
Epoch 4/150
18/18 [==============================] - 0s 11ms/step - loss: 2.9378 - mae: 1.3804
Epoch 5/150
18/18 [==============================] - 0s 11ms/step - loss: 2.9022 - mae: 1.3702
Epoch 6/150
18/18 [==============================] - 0s 11ms/step - loss: 2.8562 - mae: 1.3635
Epoch 7/150
18/18 [==============================] - 0s 11ms/step - loss: 2.8080 - mae: 1.3511
Epoch 8/150
18/18 [==============================] - 0s 11ms/step - loss: 2.8673 - mae: 1.3764
Epoch 9/150
18/18 [==============================] - 0s 11ms/step - loss: 2.9237 - mae: 1.3806
Epoch 10/150
18/18 [==============================] - 0s 11ms/step - loss: 2.7874 - mae: 1.3464
Epoch 11/150
18/18 [============================

In [20]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

15/15 [==============================] - 1s 4ms/step


array([[4.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [6.],
       [5.],
       [6.],
       [4.],
       [4.],
       [6.],
       [6.],
       [4.],
       [4.],
       [6.],
       [6.],
       [6.],
       [4.],
       [5.],
       [6.],
       [5.],
       [4.],
       [6.],
       [5.],
       [6.],
       [5.],
       [6.],
       [5.],
       [5.],
       [6.],
       [6.],
       [6.],
       [6.],
       [4.],
       [6.],
       [5.],
       [6.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],
       [6.],
       [6.],
       [5.],
       [6.],
       [5.],
       [5.],
       [4.],
       [5.],
       [6.],
       [6.],
       [6.],
       [6.],
       [6.],
       [4.],
       [6.],
       [5.],
       [5.],
       [6.],
       [5.],
       [6.],
       [5.],
       [5.],
       [5.],
       [5.],
       [5.],
       [6.],
       [5.],
       [4.],
       [6.],
       [4.],
       [6.],
       [5.],
       [6.],